Zad. 1 
Napisz program pobierające dane z Internetu.

Zadanie polega na znalezieniu legalnego źródła mediów w Internecie np. zdjęć. Program powinien posiadać pasek postępu oraz łatwy do zrozumienia system logowania błędów.

In [18]:
import requests
import os
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin, urlparse


def is_valid(url):
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)


def get_all_images(url):
    soup = bs(requests.get(url).content, "html.parser")
    urls = []
    for img in tqdm(soup.find_all("img"), "Extracting images"):
        img_url = img.attrs.get("src")
        if not img_url:
            # if img does not contain src attribute, just skip
            continue
        # make the URL absolute by joining domain with the URL that is just extracted
        img_url = urljoin(url, img_url)

        # finally, if the url is valid
        if is_valid(img_url):
            urls.append(img_url)
#     print(urls)
    return urls


def download(url, pathname, filename):
    # if path doesn't exist, make that path dir
    if not os.path.isdir(pathname):
        os.makedirs(pathname)
    # download the body of response by chunk, not immediately
    response = requests.get(url, stream=True)

    # get the total file size
    file_size = int(response.headers.get("Content-Length", 0))

    # progress bar, changing the unit to bytes instead of iteration (default by tqdm)
    progress = tqdm(response.iter_content(1024), f"Downloading {filename}", total=file_size, unit="B", unit_scale=True, unit_divisor=1024)
    with open(filename, "wb") as f:
        for data in progress.iterable:
            # write data read to the file
            f.write(data)
            # update the progress bar manually
            progress.update(len(data))


def main(url, path):
    # get all images
    imgs = get_all_images(url)
    filename = 0
    for img in imgs:
        filename += 1
        # for each img, download it
        download(img, path, f'{path}\\{filename}.jpg')
    


if __name__ == "__main__":
    url = "https://yandex.com/images/"
    path = "imgs"
    
    main(url, path)



Extracting images: 100%|███████████████████████████████████████████████████████████████████████| 22/22 [00:00<?, ?it/s]










































